In [5]:
import os
import random
import numpy as np
import torch

# Bước 1: Cố định seed (chỉnh lại số seed nếu đội muốn)

def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("Seed fixed at 42")


Seed fixed at 42


In [6]:
# Bước 2: Nạp mô hình và các tài nguyên cần thiết

import cv2
import glob
from ultralytics import YOLO

from predict import AeroEyesPredictor

# Đường dẫn mặc định giống predict.py
DATA_DIR = os.getenv(
    "DATA_DIR",
    "public_test\samples",
)
MODEL_PATH = "saved_models/yolov8l_1e.pt"

print(f"DATA_DIR = {DATA_DIR}")
print(f"MODEL_PATH = {MODEL_PATH}")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model not found at {MODEL_PATH}")

# Khởi tạo biến dùng lại ở cell sau
predictor = None

# Lấy danh sách các case (sub-folder) trong DATA_DIR
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Data directory not found at {DATA_DIR}")

subfolders = sorted([f.path for f in os.scandir(DATA_DIR) if f.is_dir()])
print(f"Found {len(subfolders)} cases.")


DATA_DIR = ../data/public_test/public_test/samples
MODEL_PATH = saved_models/yolov8l_1e.pt
Found 6 cases.


<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ACER\AppData\Local\Temp\ipykernel_22768\2030313176.py:12: SyntaxWarning: invalid escape sequence '\s'
  "public_test\samples",


In [7]:
# Bước 3: Đọc nội dung từng test case (danh sách video + thư mục ref)

cases = []
for folder_path in subfolders:
    case_name = os.path.basename(folder_path)
    video_path = os.path.join(folder_path, "drone_video.mp4")
    ref_images_path = os.path.join(folder_path, "object_images")

    if not os.path.exists(video_path):
        # Bỏ qua case không đủ dữ liệu
        continue

    cases.append(
        {
            "case_name": case_name,
            "video_path": video_path,
            "ref_images_path": ref_images_path,
        }
    )

print(f"Usable test cases: {len(cases)}")
for c in cases[:5]:
    print(c)


Usable test cases: 6
{'case_name': 'BlackBox_0', 'video_path': '../data/public_test/public_test/samples\\BlackBox_0\\drone_video.mp4', 'ref_images_path': '../data/public_test/public_test/samples\\BlackBox_0\\object_images'}
{'case_name': 'BlackBox_1', 'video_path': '../data/public_test/public_test/samples\\BlackBox_1\\drone_video.mp4', 'ref_images_path': '../data/public_test/public_test/samples\\BlackBox_1\\object_images'}
{'case_name': 'CardboardBox_0', 'video_path': '../data/public_test/public_test/samples\\CardboardBox_0\\drone_video.mp4', 'ref_images_path': '../data/public_test/public_test/samples\\CardboardBox_0\\object_images'}
{'case_name': 'CardboardBox_1', 'video_path': '../data/public_test/public_test/samples\\CardboardBox_1\\drone_video.mp4', 'ref_images_path': '../data/public_test/public_test/samples\\CardboardBox_1\\object_images'}
{'case_name': 'LifeJacket_0', 'video_path': '../data/public_test/public_test/samples\\LifeJacket_0\\drone_video.mp4', 'ref_images_path': '../da

In [8]:
# Bước 4: Thực hiện dự đoán, đo thời gian và ghi kết quả

import json
from time import time
from tqdm import tqdm

# Hàm ghi file kết quả giống predict.py

def convert(o):
    if isinstance(o, np.integer):
        return int(o)
    if isinstance(o, np.floating):
        return float(o)
    if isinstance(o, np.ndarray):
        return o.tolist()
    return o


def write_predict_file(results, output_path="output.json"):
    with open(output_path, "w") as f:
        json.dump(results, f, indent=4, default=convert)


def write_time_file(all_predicted_time, time_path="time_submission.csv"):
    # time_submission.csv: 2 cột: case_name, time_ms
    import csv

    with open(time_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["case_name", "time_ms"])  # header
        for case_name, t_ms in all_predicted_time:
            writer.writerow([case_name, t_ms])


all_results = []
all_predicted_time = []

for case in tqdm(cases, desc="Timing cases"):
    case_name = case["case_name"]
    video_path = case["video_path"]
    ref_images_path = case["ref_images_path"]

    # Nạp mô hình & tài nguyên cho từng case (theo đúng yêu cầu: từ load -> inference)
    t1 = time()
    predictor = AeroEyesPredictor(
        MODEL_PATH,
        ref_images_path,
        conf_threshold=0.001,
        color_tol=20.0,
    )

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Skip {case_name}: cannot open video")
        continue

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_idx = 0
    case_result = {"video_id": case_name, "detections": []}
    has_detections = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        box = predictor.predict_streaming(frame_rgb, frame_idx)

        if box is not None:
            if not has_detections:
                case_result["detections"].append({"bboxes": []})
                has_detections = True

            case_result["detections"][0]["bboxes"].append(
                {
                    "frame": int(frame_idx),
                    "x1": int(box[0]),
                    "y1": int(box[1]),
                    "x2": int(box[2]),
                    "y2": int(box[3]),
                }
            )

        frame_idx += 1

    cap.release()
    all_results.append(case_result)

    t2 = time()
    predicted_time_ms = int((t2 - t1) * 1000)
    all_predicted_time.append((case_name, predicted_time_ms))
    print(f"Case {case_name} time: {predicted_time_ms} ms")

# Ghi file kết quả và file thời gian
write_predict_file(all_results, output_path="output.json")
write_time_file(all_predicted_time, time_path="time_submission.csv")

print("Done timing. Results saved to output.json and time_submission.csv")


Timing cases:   0%|          | 0/6 [00:00<?, ?it/s]

Loaded 3 reference colors.


Timing cases:   0%|          | 0/6 [00:01<?, ?it/s]


KeyboardInterrupt: 